In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-hadoop'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]")\
.config("spark.driver.memory", "12g")\
.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11")\
.appName("ch31_DeepLearningLappy-docs").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "8g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "10")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [3]:
sc = spark.sparkContext

In [4]:
spark

In [5]:
# In[15]:
img_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images"
img_dir_comb = "D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\Combined30"

In [6]:
import keras
from keras.applications import InceptionV3

model = InceptionV3(weights= None, classes=2)
model.save("D:/tmp/model-full-tmp.h5")

# Signature: InceptionV3(include_top=True, weights='imagenet', 
#                        input_tensor=None, input_shape=None, pooling=None, classes=1000)
# Docstring:
# Instantiates the Inception v3 architecture.

Using TensorFlow backend.


In [7]:
from keras.layers import Activation, Dense, Flatten
from keras.models import Sequential

model = Sequential()
model.add(Flatten(input_shape=(299, 299, 3)))
model.add(Dense(2))
model.add(Activation("softmax"))
model.save("D:/tmp/model-full-tmp1.h5")  # saves to the local filesystem


In [8]:
import os
#from os import listdir
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))

In [9]:
absoluteFilePaths(img_dir + "/tulips")

<generator object absoluteFilePaths at 0x000001ED85D706D0>

In [10]:
tulips_files = [str(f) for f in absoluteFilePaths(img_dir + "/tulips")]  # make "local" file paths for images
daisy_files = [str(f) for f in absoluteFilePaths(img_dir + "/daisy")]  # make "local" file paths for images

In [11]:
tulips_files[:1]

['D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\tulips\\3498663243_42b39b4185_m.jpg']

In [12]:
daisy_files[:1]

['D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\daisy\\10555815624_dc211569b0.jpg']

In [13]:
import pyspark.ml.linalg as spla
import pyspark.sql.types as sptyp
import numpy as np

In [14]:
def CreateTrainImageUriandLabels(image_uris, label, cardinality):
  # Create image categorical labels (integer IDs)
  local_rows = []
  for uri in image_uris:
    label_inds = np.zeros(cardinality)
    label_inds[label] = 1.0
    one_hot_vec = spla.Vectors.dense(label_inds.tolist())
    _row_struct = {"uri": uri, "one_hot_label": one_hot_vec, "label": float(label)}
    row = sptyp.Row(**_row_struct)
    local_rows.append(row)

  image_uri_df = spark.createDataFrame(local_rows)
  return image_uri_df

In [15]:
label_cardinality = 2

In [16]:
tulips_uri_df = CreateTrainImageUriandLabels(tulips_files,1,label_cardinality)

In [17]:
daisy_uri_df = CreateTrainImageUriandLabels(daisy_files,0,label_cardinality)

In [18]:
tulips_uri_df.show(2,False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3498663243_42b39b4185_m.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3501996215_1c6d1a3386_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [19]:
daisy_uri_df.show(2, False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|0.0  |[1.0,0.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\daisy\10555815624_dc211569b0.jpg  |
|0.0  |[1.0,0.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\daisy\10555826524_423eb8bf71_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [20]:
#tulips_train, tulips_test, _ = tulips_uri_df.randomSplit([0.005, 0.005, 0.99]) 
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
tulips_train, tulips_test = tulips_uri_df.randomSplit([0.8, 0.2]) 

In [21]:
#daisy_train, daisy_test, _ = daisy_uri_df.randomSplit([0.005, 0.005, 0.99])    
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
daisy_train, daisy_test = daisy_uri_df.randomSplit([0.8, 0.2]) 

In [22]:
train_df = tulips_train.unionAll(daisy_train).cache()
test_df = tulips_test.unionAll(daisy_test).cache()

In [23]:
train_df.show(3, False)
test_df.show(3, False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3501996215_1c6d1a3386_n.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3502085373_edc2c36992_n.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3502251824_3be758edc6_m.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows

+-----+-------------+----------------------------------------------------------

In [24]:
import PIL.Image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from sparkdl.estimators.keras_image_file_estimator import KerasImageFileEstimator

In [25]:
def load_image_from_uri(local_uri):
  img = (PIL.Image.open(local_uri).convert('RGB').resize((299, 299), PIL.Image.ANTIALIAS))
  img_arr = np.array(img).astype(np.float32)
  img_tnsr = preprocess_input(img_arr[np.newaxis, :])
  return img_tnsr

In [26]:
train_df.select("uri").first()[0]

'D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\tulips\\3501996215_1c6d1a3386_n.jpg'

In [27]:
test_load = load_image_from_uri(train_df.select("uri").first()[0])

In [28]:
test_load.shape

(1, 299, 299, 3)

In [29]:
# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
# train_df = train_df.repartition(100)
# test_df = test_df.repartition(100)

In [38]:
estimatorT = KerasImageFileEstimator( inputCol="uri",
                                     outputCol="prediction",
                                     labelCol="one_hot_label",
                                     imageLoader=load_image_from_uri,
                                     kerasOptimizer='adam',
                                     kerasLoss='categorical_crossentropy', # why categorical_crossentropy
                                     modelFile="D:/tmp/model-full-tmp.h5", # local file path for model
                                     kerasFitParams = {"batch_size": 32, "verbose": 0}
                                   ) 

# estimator = KerasImageFileEstimator(inputCol="imageUri",
#                                         outputCol="name_of_result_column",
#                                         labelCol="categoryVec",
#                                         imageLoader=load_image_and_process,
#                                         kerasOptimizer="adam",
#                                         kerasLoss="categorical_crossentropy",
#                                         kerasFitParams={"epochs": 5, "batch_size": 64},
#                                         modelFile="path_to_my_model.h5")

#     transformers = estimator.fit(image_dataset)
# Init docstring:
# __init__(self, inputCol=None, outputCol=None, 
#          outputMode="vector", labelCol=None,
#          modelFile=None, imageLoader=None, 
#          kerasOptimizer=None, kerasLoss=None,
#          kerasFitParams=None)

In [39]:
train_df.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|   23|
|  1.0|   19|
+-----+-----+



In [40]:
fittedT = estimatorT.fit(train_df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 495, localhost, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:428)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:421)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:252)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIt

In [41]:
estimator = KerasImageFileEstimator( inputCol="uri",
                                     outputCol="prediction",
                                     labelCol="one_hot_label",
                                     imageLoader=load_image_from_uri,
                                     kerasOptimizer='adam',
                                     kerasLoss='categorical_crossentropy', # why categorical_crossentropy
                                     modelFile="D:/tmp/model-full-tmp1.h5" # local file path for model
                                   ) 

In [47]:
fitted = estimator.fit(train_df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 37.0 failed 1 times, most recent failure: Lost task 0.0 in stage 37.0 (TID 529, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\client\session.py", line 1361, in _do_call
    return fn(*args)
  File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\client\session.py", line 1340, in _run_fn
    target_list, status, run_metadata)
  File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 516, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.InternalError: Blas GEMM launch failed : a.shape=(32, 268203), b.shape=(268203, 2), m=32, n=2, k=268203
	 [[Node

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [43]:
paramGrid = (
  ParamGridBuilder()
  .addGrid(estimator.kerasFitParams, [{"batch_size": 16, "verbose": 0}])
  .build()
)

In [44]:
# paramGrid = (
#   ParamGridBuilder()
#   .addGrid(estimator.kerasFitParams, [{"batch_size": 32, "verbose": 0},
#                                       {"batch_size": 64, "verbose": 0}])
#   .build()
# )

In [45]:
bc = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label" )
cv = CrossValidator(estimator=estimator, estimatorParamMaps=paramGrid, evaluator=bc, numFolds=2)

In [46]:
cvModel = cv.fit(train_df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 512, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\client\session.py", line 1361, in _do_call
    return fn(*args)
  File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\client\session.py", line 1340, in _run_fn
    target_list, status, run_metadata)
  File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 516, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.InternalError: Blas GEMM launch failed : a.shape=(16, 268203), b.shape=(268203, 2), m=16, n=2, k=268203
	 [[Node

In [ ]:
# cvModel = cv.fit(train_df)

# #
# File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\keras\engine\training_utils.py", line 137, in standardize_input_data
#     str(data_shape))
# ValueError: Error when checking target: expected predictions to have shape (1000,) but got array with shape (2,)
